In [1]:
!pip install -i https://test.pypi.org/simple/ anguilla --upgrade

Looking in indexes: https://test.pypi.org/simple/
     |████████████████████████████████| 354 kB 321 kB/s eta 0:00:01
  Attempting uninstall: anguilla
    Found existing installation: anguilla 0.0.11
    Uninstalling anguilla-0.0.11:
      Successfully uninstalled anguilla-0.0.11


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pathlib
import anguilla

import anguilla.hypervolume as hv

from anguilla.dominance import NonDominatedSet2D, non_dominated_sort
from anguilla.fitness import benchmark

print(anguilla.__version__)

0.0.14


In [3]:
PAIRS = [
    ('ZDT1', 2),
    ('ZDT2', 2),
    ('ZDT3', 2),
    ('ZDT4', 2),
    ('ZDT6', 2),
    ('IHR1', 2),
    ('IHR2', 2),
    ('IHR3', 2),
    ('IHR4', 2),
    ('IHR6', 2),
    ('ELLI1', 2),
    ('ELLI2', 2),
    ('CIGTAB1', 2),
    ('CIGTAB2', 2),
    ('DTLZ1', 3),
    ('DTLZ2', 3),
    ('DTLZ3', 3),
    ('DTLZ4', 3),
    ('DTLZ5', 3),
    ('DTLZ6', 3),
    ('DTLZ7', 3),
    ('GELLI', 3)
]

ALGOS = [
    '(100+1)-MO-CMA-ES-I',
    '(100+1)-MO-CMA-ES-P',
    '(100+100)-MO-CMA-ES-I',
    '(100+100)-MO-CMA-ES-P'
]

In [4]:
class NonDominatedSetKD:
    def __init__(self):
        self.union = []
    
    def insert(self, points):
        self.union.append(points)
        
    @property
    def upper_bound(self):
        points = np.vstack(self.union)
        ranks, _ = non_dominated_sort(points, 1)
        nondominated = points[ranks == 1]
        reference = np.max(nondominated, axis=0)
        return reference

In [5]:
def check_exists(basepath, func):
    basepath = pathlib.Path(basepath)
    paths = list(basepath.glob('{}*.fitness.csv'.format(func)))
    return len(paths) != 0

def get_fn_glob(basepath, func, n_evaluations):
    basepath = pathlib.Path(basepath)
    paths = basepath.glob('{}*_{}.fitness.csv'.format(func, n_evaluations))
    return paths

def get_fn_algo_glob(basepath, func, algo, n_evaluations):
    basepath = pathlib.Path(basepath)
    paths = basepath.glob('{}_{}_*_{}.fitness.csv'.format(func, algo, n_evaluations))
    return paths

def get_fn_reference_point(basepath, func, n_objs, n_evaluations):
    paths = get_fn_glob(basepath, func, n_evaluations)
    point_set = NonDominatedSet2D() if n_objs == 2 else NonDominatedSetKD()
    for path in paths:
        points = np.genfromtxt(path, delimiter=',')
        point_set.insert(points)
    reference = point_set.upper_bound + 1.0
    return reference

def compute_fn_hv_indicators(reference, basepath, func, algo, n_objs, n_evaluations):
    paths = get_fn_algo_glob(basepath, func, algo, n_evaluations)
    indicators = []
    for path in paths:
        points = np.genfromtxt(path, delimiter=',')
        indicator = hv.calculate(points, reference)
        indicators.append(indicator)
    return np.array(indicators)

def compute_summaries(basepath, n_evaluations):
    refs = {}
    for func, n_objs in PAIRS:
        if check_exists(basepath, func):
            reference = get_fn_reference_point(basepath, func, n_objs, n_evaluations)
            for algo in ALGOS:
                indicators = compute_fn_hv_indicators(reference, basepath, func, algo, n_objs, n_evaluations)
                print(indicators)
                median_indicator = np.median(indicators)
                print(func, median_indicator)

In [ ]:
compute_summaries('data/shark', 50000)

[3.66202274 2.92951051 3.66202103 3.66206296 3.66198308 3.662058
 3.66175408 3.66202605 3.66205544 3.6620237  3.66205727 3.66201342
 3.66204168 3.66201649 3.66197988 3.66206634 3.66198587 3.66200563
 3.66206131 3.66197247 3.66205321 3.60696632 3.66199864 3.66198629
 3.66199583]
ZDT1 3.6620164853866353
[3.66201133 3.66207088 3.66206088 3.66204021 3.66206983 3.66207276
 3.66207718 3.6620305  3.6620779  3.66207825 3.6620736  3.66207675
 3.66203008 3.662085   3.66208078 3.66207185 3.66207815 3.6620722
 3.66207711 3.66206925 3.66208268 3.66207463 3.66208191 3.66206711
 3.66206032]
ZDT1 3.6620727647979647
[3.66191336 3.66191352 3.66194813 3.66192665 3.66192942 3.6618957
 3.66190259 3.66188494 3.66192682 3.66194509 3.66194691 3.66191871
 3.66192217 3.66193608 3.6619495  3.6619062  3.66194141 3.66187656
 3.66190714 3.66192861 3.66188304 3.66193399 3.66192457 3.66194181
 3.66193052]
ZDT1 3.661926652738308
[3.66190176 3.66192408 3.661948   3.66194872 3.6618871  3.66195525
 3.66196756 3.66188633 